In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# Cargar los datos
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')
productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")

# Crear DataFrame principal
df = sellin.merge(productos, on="product_id", how="left").merge(stocks, on=["product_id", "periodo"], how="left")
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

# Sumarizar ventas por periodo y producto
ventas = df.groupby(['periodo', 'product_id']).agg({'tn':'sum'}).reset_index()
ventas['periodo_dt'] = pd.to_datetime(ventas['periodo'].astype(str), format='%Y%m')

# Generar toda la historia de cada producto
vida_producto = ventas.groupby('product_id')['periodo_dt'].agg(['min','max']).reset_index()
all_periods = []
for _, row in vida_producto.iterrows():
    pid = row['product_id']
    periods = pd.date_range(row['min'], row['max'], freq='MS')
    for p in periods:
        all_periods.append((pid, p))
df_full = pd.DataFrame(all_periods, columns=['product_id','periodo_dt'])
ventas = ventas[['product_id','periodo_dt','tn']]
df_pp = df_full.merge(ventas, on=['product_id','periodo_dt'], how='left')
df_pp['tn'] = df_pp['tn'].fillna(0)

# Preparar target
df_pp['periodo'] = df_pp['periodo_dt'].dt.strftime('%Y%m').astype(int)
df_pp['tn_target'] = df_pp.groupby('product_id')['tn'].shift(-2)

# Feature engineering básico
df_pp['month'] = df_pp['periodo_dt'].dt.month
df_pp['quarter'] = df_pp['periodo_dt'].dt.quarter
df_pp['year'] = df_pp['periodo_dt'].dt.year
df_pp['tn_lag1'] = df_pp.groupby('product_id')['tn'].shift(1)
df_pp['tn_lag2'] = df_pp.groupby('product_id')['tn'].shift(2)
df_pp['stock_lag1'] = df_pp.groupby('product_id')['tn'].shift(1)

# Filtrar filas válidas (sin NaN en target)
train_data = df_pp.dropna(subset=['tn_target'])

# Definir predicción con AutoGluon
predictor = TabularPredictor(label='tn_target', eval_metric='rmse').fit(
    train_data.drop(columns=['periodo_dt', 'tn']),
    time_limit=600,  # Limitar el tiempo de entrenamiento
    presets='best_quality'  # Mejor calidad posible
)

In [ ]:
# Preparar datos para predicción (periodo 201912)
X_test = df_pp[df_pp['periodo'] == 201912].drop(columns=['periodo_dt', 'tn_target', 'tn'], errors='ignore')
productos_ids = productos_a_predecir['product_id'].unique()
X_test = X_test[X_test['product_id'].isin(productos_ids)]

# Alinear con features
feature_columns = [col for col in predictor.feature_metadata.get_features()]

# Agrupar por product_id (promediar si hay duplicados)
X_test_grouped = X_test.groupby('product_id')[feature_columns].mean().reset_index()

# Predecir
preds = predictor.predict(X_test_grouped)

# Guardar resultados
result = pd.DataFrame({'product_id': X_test_grouped['product_id'], 'tn': preds})
result.to_csv('./kaggle/autogluon_optimized.csv', index=False)
print("✅ Predicciones guardadas en ./kaggle/autogluon_optimized.csv")
